### Mapas coropléticos (*Choropleth Maps*)

Nos mapas coropléticos as diferentes áreas están coloreadas ou representadas con diferentes esquemas en relación a certos valores estatísticos asociados a elas.

O primeiro mapa "mapa coropleth", coñecido foi creado en 1826 polo barón Pierre Charles Dupin, que mostra a dispoñibilidade de educación básica en Francia por departamentos. 

Mais abaixo podes ver un dos primeiros mapas de Charles Dupin.

Fonte: https://gl.wikipedia.org/wiki/Mapa_coropl%C3%A9tico 

In [ ]:
from IPython import display 
display.Image(url="https://upload.wikimedia.org/wikipedia/commons/5/52/Carte_de_France_des_crimes_contres_les_personnes.jpg", width=400, height=400)

Este notebook está baseado nun titorial de M.Rake Linggar, publicado en TDS [(enlace)](#https://towardsdatascience.com/a-beginners-guide-to-create-a-cloropleth-map-in-python-using-geopandas-and-matplotlib-9cc4175ab630)

Neste notebook farase un mapa coroplético que represente o número de cidadades e rexións en cada provincia de Indonesia. 

Utilizaranse dúas fontes de datos:
- Información xeográfica sobre o territorio de Indonesia (Shapefile)
- Información sobre as provincias de Indonesia (scrapping desde a [Wikipedia](#https://en.wikipedia.org/wiki/Provinces_of_Indonesia))

In [ ]:
# Importar as librarías necesarias
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
# Cargamos os datos do mapa e comprobamos os datos
fp = "../../datasets/choropleth/IDN_adm/IDN_adm1.shp"
map_df = gpd.read_file(fp)
map_df.head()

In [ ]:
# Pintamos o mapa directamente desde o dataframe
map_df.plot(figsize=(15,30))

In [ ]:
# Cargamos os datos das provincias
province = pd.read_csv("../../datasets/choropleth/data_province.csv", sep=";")
province.head()

In [ ]:
# Podemos comprobar se ambos os dataframes teñen o mesmo número de filas
rmap, cmap = map_df.shape
rpro, cpro = province.shape
print(f'Filas do mapa: {rmap}')
print(f'Filas en provincias: {rpro}')

In [ ]:
# Para representar os datos sobre o mapa debemos crear un único dataframe

# Podemos facer un merge dun dataframe e un geodataframe ;-)

merged = map_df.merge(province, how='left', left_on="NAME_1", right_on="province")
merged = merged[['province', 'geometry', 'population_2015', 'area_km2', 'population_density_per_km2', \
                'cities_regencies', 'cities', 'regencies']]

merged.head()

In [ ]:
# Indicamos a variable que queremos representar
variable = 'cities_regencies'

# Fixamos o rango que tomarán os valores
vmin, vmax = 0, 50

# Definimos a figura para o gráfico

# A 'figura' é como o 'contedor' onde vamos pintar unha ou varias gráficas
# Cada gráfica pode ter os seus propios eixos (axes) ou poden compartilos

fig, ax = plt.subplots(1, figsize=(30, 10))

# Non pintamos os eixos
ax.axis('off')

# Engadimos un título
ax.set_title('Número de cidades e rexións', fontdict={'fontsize': '25', 'fontweight' : '3'})

# Lenda: barra
sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))

# empty array for the data range
sm.set_array([]) # or alternatively sm._A = []. Not sure why this step is necessary, but many recommends it

# Engadir a barra á figura
fig.colorbar(sm)

# Crear mapa
merged.plot(column=variable, cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8')
 

In [ ]:
# Podemos facer algunhas variacións sobre o mapa:
# - engadir os nomes das provincias
# - engadir unha nota sobre a fonte dos datos
# - mudar a barra a posición horizontal

In [ ]:
variable = 'cities_regencies'
vmin, vmax = 0, 50
fig, ax = plt.subplots(1, figsize=(30, 10))
ax.axis('off')
ax.set_title('Número de cidades e rexións', fontdict={'fontsize': '25', 'fontweight' : '3'})

# Engadir unha nota ao pé 
ax.annotate('Fonte: Wikipedia - https://en.wikipedia.org/wiki/Provinces_of_Indonesia',xy=(0.2, .05),  \
            xycoords='figure fraction', fontsize=12, \
            color='#555555')

# Lenda: barra
sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm.set_array([])
fig.colorbar(sm, orientation="horizontal", fraction=0.036, pad=0.1, aspect = 30)

# Pintar a gráfica
merged.plot(column=variable, cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8')

# Engadir etiquetas cos nomes das provincias
# Primeiro extrae un punto representativo de cada polígono

merged['coords'] = merged['geometry'].apply(lambda x: x.representative_point().coords[:])
merged['coords'] = [coords[0] for coords in merged['coords']]

# Itera as liñas do dataset e vai engadindo as etiquetas co nome da provincia
for idx, row in merged.iterrows():
    plt.annotate(row['province'], xy=row['coords'],horizontalalignment='center')

In [ ]:
# Podemos salvar a figura resultante a un ficheiro
fig.savefig('mapa_indonesia.png', dpi=300)

In [ ]:
# O Dataframe ten outras variables coas que poderiamos pintar o mapa
merged.head(3)

In [ ]:
# Pinta un mapa segundo número de cidades
# - Cambia o esquema de cores #https://matplotlib.org/stable/tutorials/colors/colormaps.html
# - Cambia o título
# - Ollo ao rango de valores

In [ ]:
# Pinta un mapa segundo número a densidade de poboación
# - Cambia o esquema de cores
# - Cambia o título